This code was used to align and clip all rasters.
For this we use a base layer, in our case we use the projected slopes layer in 3035 with a pixel size of 100 meters

In [1]:
import rasterio
from rasterio.enums import Resampling
import os
import glob
from tqdm import tqdm 
import numpy as np
import georasters as gr
import numpy as np
import os
import rasterio
from rasterio.plot import show


# Get the current directory of the script
base_path = os.getcwd()
#// Aiguestortes
#// Ordesa
#// Peneda
#// Guadarrama
#// Picos
#// SierraNieves
#// SierraNevada
#// Teide


parque = 'SierraNevada'

# Relative paths
input_folder_relative = f"C:/Users/carlo/Documents/EarthCul/Remote sensing/results/{parque}"
output_folder = f"C:/Users/carlo/Documents/EarthCul/Post processing/results/{parque}"#os.path.join(base_path, f"../Remote sensing/results/{parque}")
base_raster_path = f"C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/BaseLayers/{parque}/Slope.tif"
input_files = glob.glob(os.path.join(base_path, input_folder_relative, '*.tif'))


with rasterio.open(base_raster_path) as base_src:
    for input_file in tqdm(input_files, desc="Aligning rasters"):
        with rasterio.open(input_file) as src:
            data = src.read(1)
            profile = src.profile

            # Reproject/resample the input raster to match the base raster
            data_resampled = np.zeros_like(base_src.read(1))
            rasterio.warp.reproject(
                source=data,
                destination=data_resampled,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=base_src.transform,
                dst_crs=base_src.crs,
                resampling=Resampling.nearest
            )

            # Update the profile of the input raster to match the base raster's metadata
            profile_resampled = src.profile.copy()
            profile_resampled.update({
                'transform': base_src.transform,
                'height': data_resampled.shape[0],
                'width': data_resampled.shape[1],
                'crs': base_src.crs
            })
            input_filename = os.path.basename(input_file)

            # Save
            output_path = os.path.join(output_folder, input_filename)
            with rasterio.open(output_path, 'w', **profile_resampled) as dst:
                dst.write(data_resampled, 1)

Aligning rasters:   0%|          | 0/1 [00:00<?, ?it/s]


RasterioIOError: Attempt to create new tiff file 'C:/Users/carlo/Documents/EarthCul/Post processing/results/SierraNevada/viewshed.tif' failed: No such file or directory

Recortar los raster usando una mascara 

In [9]:
import os
import glob
import rasterio
from rasterio.mask import mask
import geopandas as gpd

# Definir la ruta de la carpeta que contiene los rasters a recortar y la capa de máscara
carpeta_rasters = f"C:/Users/carlo/Documents/EarthCul/Post processing/results/{parque}"#os.path.join(base_path, "../Teledeteccion_variables/Alineados")
mascara_path = f"C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/BaseLayers/{parque}/AOI_3035.shp"

# Cargar la capa de máscara
mask_geom = gpd.read_file(mascara_path)
mask_shapes = mask_geom.geometry

# Iterar sobre todos los archivos raster en la carpeta
for raster_path in glob.glob(os.path.join(carpeta_rasters, "*.tif")):
    # Cargar el raster
    with rasterio.open(raster_path) as src:
        raster_crs = src.crs
        raster_meta = src.meta.copy()

        # Recortar el raster usando la capa de máscara
        masked_data, masked_transform = mask(src, mask_shapes, crop=True)
        masked_meta = src.meta.copy()

        # Actualizar los metadatos del raster recortado
        masked_meta.update({
            "height": masked_data.shape[1],
            "width": masked_data.shape[2],
            "transform": masked_transform,
            "nodata": -9999  # Opcional: ajusta el valor de nodata según sea necesario
        })

        # Guardar el raster recortado
        nombre_raster = os.path.basename(raster_path)
        output_path = os.path.join(os.path.join(base_path, f"results/{parque}/clipped"), nombre_raster)
        with rasterio.open(output_path, 'w', **masked_meta) as dst:
            dst.write(masked_data)



## 1) Resample and align
With this code you can resample all the tiffs from subfolders and save them in the same subfolders

In [2]:
import os
import glob
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling
# from rasterio.enums import Resampling
from tqdm import tqdm

# Obtener el directorio actual del script
base_path = os.getcwd()
# parque = 'Aiguestortes'

# Definir parques
parques = ['Aiguestortes', 'Ordesa', 'Peneda', 'Guadarrama', 'Picos', 'SierraNieves', 'SierraNevada', 'Teide']

# Ruta del directorio de entrada y salida
input_base_folder = "../Remote sensing/results"
output_base_folder = "../Post processing/results/Aligned"

# Ruta de la capa base
base_raster_path = f"C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/BaseLayers/{parque}/Slope.tif"

for parque in parques:
    input_folder = os.path.join(base_path, input_base_folder, parque)
    output_folder = os.path.join(base_path, output_base_folder, parque)

    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)

    # Buscar archivos TIF en la carpeta de entrada y sus subcarpetas
    input_files = glob.glob(os.path.join(input_folder, '**/*.tif'), recursive=True)

    # Ruta de la capa base para este parque
    base_raster_path_parque = base_raster_path.format(parque=parque)

    with rasterio.open(base_raster_path_parque) as base_src:
        for input_file in tqdm(input_files, desc=f"Aligning rasters for {parque}"):
            with rasterio.open(input_file) as src:
                data = src.read(1)
                profile = src.profile

                # Reproject/resample the input raster to match the base raster
                data_resampled = np.zeros_like(base_src.read(1))
                rasterio.warp.reproject(
                    source=data,
                    destination=data_resampled,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=base_src.transform,
                    dst_crs=base_src.crs,
                    resampling=Resampling.nearest
                )

                # Update the profile of the input raster to match the base raster's metadata
                profile_resampled = src.profile.copy()
                profile_resampled.update({
                    'transform': base_src.transform,
                    'height': data_resampled.shape[0],
                    'width': data_resampled.shape[1],
                    'crs': base_src.crs
                })
                input_filename = os.path.relpath(input_file, input_folder)

                # Save
                output_path = os.path.join(output_folder, input_filename)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                with rasterio.open(output_path, 'w', **profile_resampled) as dst:
                    dst.write(data_resampled, 1)


Aligning rasters for Aiguestortes:   0%|          | 0/1 [00:00<?, ?it/s]

Aligning rasters for Teide: 100%|██████████| 1/1 [00:00<00:00,  6.98it/s]


## 2) Clip using a shapefile from the grid layer 


Aplicar para carpetas y subcarpetas

## Recortar los archivos 

In [3]:
import os
from osgeo import gdal

# Definir la lista de parques
parques = ['Aiguestortes', 'Ordesa', 'Peneda', 'Guadarrama', 'Picos', 'SierraNieves', 'SierraNevada', 'Teide']

# Ruta al shapefile de máscara
base_mask_folder = 'C:/Users/carlo/Documents/EarthCul/OSM/2) Derived metrics/BaseLayers'

# Carpeta base para los rasters de entrada y salida
base_input_folder = 'C:/Users/carlo/Documents/EarthCul/Post processing/results/Aligned'
base_output_folder = 'C:/Users/carlo/Documents/EarthCul/Post processing/results/Clipped'

# Función para copiar la estructura de carpetas de entrada a salida
def copy_folder_structure(input_folder, output_folder):
    for dirpath, dirnames, filenames in os.walk(input_folder):
        structure = os.path.join(output_folder, os.path.relpath(dirpath, input_folder))
        os.makedirs(structure, exist_ok=True)

# Iterar sobre los parques
for parque in parques:
    input_parque_folder = os.path.join(base_input_folder, parque)
    output_parque_folder = os.path.join(base_output_folder, parque)
    mask_shapefile = os.path.join(base_mask_folder, parque, 'AOI_3035.shp')

    # Crear la carpeta de salida del parque si no existe
    os.makedirs(output_parque_folder, exist_ok=True)

    # Copiar la estructura de carpetas del parque de entrada a salida
    copy_folder_structure(input_parque_folder, output_parque_folder)

    # Función para recortar rasters
    def recortar_raster(input_raster, output_raster):
        gdal.Warp(output_raster, input_raster, cutlineDSName=mask_shapefile, cropToCutline=True, dstNodata=-9999)

    # Iterar sobre todos los archivos en la carpeta de entrada del parque
    for dirpath, dirnames, filenames in os.walk(input_parque_folder):
        for filename in filenames:
            if filename.endswith('.tif'):
                input_raster = os.path.join(dirpath, filename)
                output_raster = os.path.join(output_parque_folder, os.path.relpath(input_raster, input_parque_folder))
                
                # Recortar el raster usando gdal.Warp
                recortar_raster(input_raster, output_raster)


gdal.Warp(output_raster, input_raster, cutlineDSName=mask_shapefile, cropToCutline=True, dstNodata=-9999)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000002B3C47C1110> >